In [75]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-2aiw12p1-cifar-10/x_test.csv
/kaggle/input/2023-2aiw12p1-cifar-10/train.csv
/kaggle/input/2023-2aiw12p1-cifar-10/sample_submit.csv


In [76]:
import random
import torch
import torch.nn as nn
import numpy as np
import os

random_seed= 42
random.seed(random_seed)
np.random.seed(random_seed)
os.environ["PYTHONHASHSEED"] = str(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(False)

In [77]:
train = pd.read_csv("/kaggle/input/2023-2aiw12p1-cifar-10/train.csv")
test = pd.read_csv("/kaggle/input/2023-2aiw12p1-cifar-10/x_test.csv")
submit = pd.read_csv("/kaggle/input/2023-2aiw12p1-cifar-10/sample_submit.csv")

In [78]:
x_train = train.drop(['Label'], axis=1)
y_train = train['Label']

x_test = test

In [79]:
x_train = torch.from_numpy(np.array(x_train).astype('float32'))
y_train = torch.from_numpy(np.array(y_train))

x_test = torch.from_numpy(np.array(x_test).astype('float32'))

In [80]:
x_train = x_train.reshape(-1, 32, 32, 3)
x_train = x_train.permute(0, 3, 1, 2)
print(x_train.shape)

x_test = x_test.reshape(-1, 32, 32, 3)
x_test = x_test.permute(0, 3, 1, 2)
print(x_test.shape)

print(y_train.shape)

torch.Size([50000, 3, 32, 32])
torch.Size([10000, 3, 32, 32])
torch.Size([50000])


In [81]:
from torch.utils.data import TensorDataset
train = TensorDataset(x_train, y_train)
# test = TensorDataset(x_test)

In [82]:
train_dataset, val_dataset = torch.utils.data.random_split(train, [0.9, 0.1])

In [83]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=True)

full_train_dataloader = DataLoader(train, batch_size=128, shuffle=True)
test_dataloader = DataLoader(x_test, batch_size=1, shuffle=False)

In [84]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.input_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3), stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=(2,2))
        self.dropout1 = nn.Dropout(p=0.25)
        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.flatten = nn.Flatten()
        self.hidden_layer3 = nn.Sequential(
            nn.Linear(in_features=64*8*8, out_features=512),
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        self.dropout2 = nn.Dropout(p=0.5)
        self.classifier = nn.Linear(in_features=512, out_features=10)
    
    def forward(self, input):
        x = self.input_layer(input)
        x = self.hidden_layer1(x)
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.hidden_layer2(x)
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.flatten(x)
        x = self.hidden_layer3(x)
        x = self.dropout2(x)
        x = self.classifier(x)
        return x

In [85]:
from tqdm import tqdm

In [86]:
model = CNN().cuda()

optim = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [87]:
for epoch in range(50):
    train_acc = 0
    train_cost = 0
    model.train()
    for x, y in tqdm(train_dataloader):
        x = x.cuda()
        y = y.cuda()
        
        pred = model(x)
        cost = loss_fn(pred, y)
        
        optim.zero_grad()
        cost.backward()
        optim.step()
        
        train_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1)==y).sum().item()
        train_cost += cost.item()
    
    with torch.no_grad():
        val_acc=0
        val_cost=0
        model.eval()
        for x, y in val_dataloader:
            x = x.cuda()
            y = y.cuda()
            
            pred = model(x)
            cost = loss_fn(pred, y)
            
            val_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1)==y).sum().item()
            val_cost += cost.item()
            
    print("epoch :", epoch, " train_acc :", train_acc/len(train_dataloader.dataset), " val_acc :", val_acc/len(val_dataloader.dataset))

100%|██████████| 352/352 [00:05<00:00, 63.54it/s]


epoch : 0  train_acc : 0.5532444444444444  val_acc : 0.6586


100%|██████████| 352/352 [00:05<00:00, 63.67it/s]


epoch : 1  train_acc : 0.6848888888888889  val_acc : 0.7254


100%|██████████| 352/352 [00:05<00:00, 63.38it/s]


...

epoch : 47  train_acc : 0.9650666666666666  val_acc : 0.8556


100%|██████████| 352/352 [00:05<00:00, 63.26it/s]


epoch : 48  train_acc : 0.965  val_acc : 0.8386


100%|██████████| 352/352 [00:05<00:00, 63.14it/s]


epoch : 49  train_acc : 0.9670666666666666  val_acc : 0.8506


In [96]:
model = CNN().cuda()

optim = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in tqdm(range(50)):
    train_acc = 0
    model.train()
    for x, y in full_train_dataloader:
        x = x.cuda()
        y = y.cuda()
        
        pred = model(x)
        cost = loss_fn(pred, y)
        
        optim.zero_grad()
        cost.backward()
        optim.step()
        
        train_acc += (torch.argmax(torch.softmax(pred, dim=1), dim=1)==y).sum().item()
#     print("train_acc :", train_acc/len(full_train_dataloader.dataset))

  2%|▏         | 1/50 [00:06<04:56,  6.04s/it]

train_acc : 0.55272


  4%|▍         | 2/50 [00:12<04:49,  6.03s/it]

train_acc : 0.68496


  6%|▌         | 3/50 [00:18<04:42,  6.02s/it]

...

train_acc : 0.96106


 98%|█████████▊| 49/50 [04:54<00:06,  6.02s/it]

train_acc : 0.96312


100%|██████████| 50/50 [05:00<00:00,  6.01s/it]

train_acc : 0.9641


In [97]:
model.eval()
final_result=[]
with torch.no_grad():
    for x in tqdm(test_dataloader):
        x = x.cuda()
        pred = model(x)
        
        final_result.append((torch.argmax(torch.softmax(pred, dim=1), dim=1)).item())

100%|██████████| 10000/10000 [00:10<00:00, 960.82it/s]


In [98]:
submit['Label'] = final_result
submit.to_csv("w12p1_submission.csv", index=False)